In [1]:
from alpaca_farm.models import reward_model
from alpaca_farm.inference.decode import load_model_and_tokenizer_for_inference 
from alpaca_farm.inference.score import score_sequences_with_huggingface_given_model
from alpaca_farm import utils

from transformers import AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
import torch
import os

/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda120.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 120
CUDA SETUP: Loading binary /home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda120.so...


/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/prasann/miniconda3/envs/aplora did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
n_gpus = torch.cuda.device_count()
max_memory = f'{46000}MB'
max_memory = {i: max_memory for i in range(n_gpus)}
device_map = "auto"

# if we are in a distributed setting, we need to set the device map and max memory per device
if os.environ.get('LOCAL_RANK') is not None:
    local_rank = int(os.environ.get('LOCAL_RANK', '0'))
    device_map = {'': local_rank}
    max_memory = {'': max_memory[local_rank]}
# normal llama: 
# mpath = "/home/prasann/Projects/tfr-decoding/llama/llama"
# PPO: 
mpath = "/home/prasann/Projects/tfr-decoding/apfarm_models/ppo-human"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=mpath,
    load_in_4bit=True,
    device_map=0,
    max_memory=max_memory,
    #low_cpu_mem_usage=False,
    torch_dtype=torch.float32,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float32,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)

In [4]:
tok = LlamaTokenizer.from_pretrained(mpath)

In [6]:
prompt = "Give a long and detailed response that answers the question. \n Question: Why can't penguins fly? \nResponse: "
inps = tok(prompt, return_tensors='pt').to(model.device)

In [8]:
tok.batch_decode(model.generate(**inps, max_new_tokens=200, do_sample=True))

["<s> Give a long and detailed response that answers the question. \n Question: Why can't penguins fly? \nResponse: \nPenguins are unable to fly due to their body structure. Their wings are too heavy and their body is not warm enough to generate enough energy for flight. Penguins also lack the trait of flexibility needed to make tight turns in flight, making it impossible for them to maintain flight for long periods of time. Additionally, penguins have adapted to live in the cold climate of the Antarctic, and warm body temperature is required for flight. Lastly, penguins lack the extra layer of body fat that helps to provide buoyancy in the air, making it difficult for them to stay afloat while attempting to fly.  In short, penguins' bodies are simply not designed for flight.</s>"]

In [5]:
model = reward_model.RewardModel.from_pretrained(
    "/home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/",
    load_in_4bit=True,
    device_map=0,
    max_memory=max_memory,
    #low_cpu_mem_usage=False,
    torch_dtype=torch.float32,
    flash_attn=False,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float32,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)

One of the `_name_or_path` to the root node is not a pretrained model name or local path on disk. This may be due to copying checkpoints across machines. Falling back to hardcoded rule to figure out the pretrained model based on config.json.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.85s/it]


In [4]:
rm, tokenizer = load_model_and_tokenizer_for_inference(
    model_name_or_path="/home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/",
    model_cls=reward_model.RewardModel,
    cache_dir=None,
    model_kwargs=dict(
        torch_dtype=utils.convert_str_dtype_to_torch_dtype(None),
        flash_attn=False,
    ),
)

Loading model for inference: /home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/
One of the `_name_or_path` to the root node is not a pretrained model name or local path on disk. This may be due to copying checkpoints across machines. Falling back to hardcoded rule to figure out the pretrained model based on config.json.


Using 4bit quantization


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.92s/it]


NotImplementedError: Cannot copy out of meta tensor; no data!

In [9]:
def scoreseq(inp, ans):
    # TODO make a batched version if necessary
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:{response}"
    seq = template.format(instruction=inp, response=ans)
    return score_sequences_with_huggingface_given_model(model, tok, [seq], per_device_batch_size=1)

In [10]:
scoreseq("Why can't penguins fly?", "Penguins are unable to fly due to their body structure. Their wings are too heavy and their body is not warm enough to generate enough energy for flight. Penguins also lack the trait of flexibility needed to make tight turns in flight, making it impossible for them to maintain flight for long periods of time. Additionally, penguins have adapted to live in the cold climate of the Antarctic, and warm body temperature is required for flight. Lastly, penguins lack the extra layer of body fat that helps to provide buoyancy in the air, making it difficult for them to stay afloat while attempting to fly.  In short, penguins' bodies are simply not designed for flight.")
# 1.727 from normal RM 

Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None
evaluating rewards for batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/it]


[0.7752610445022583]